# Objective

This is the template for how I processed and plotted MS2- and PP7- fluorescent intensity data for Blake & Eisen 2022. The workflow here consists of:

* extracting fluorescence intensity values for each time point from the Spots.mat files
* saving these values for a given embryo as a variable
* plotting the data in a histogram or strip plot (fluorescence intensity for a given sex over time)

At the end, I also explain how I plotted particle intensity traces for _PP7-btd_

In [2]:
#Import necessary packages
import scipy.io
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
%matplotlib inline 

In [ ]:
#Establish a project directory where your figures will save
ProjectDirectory = 'filepath'

Next, I linked variables representing individual embryos to their relevant Spots.mat file

In [ ]:
male1 = scipy.io.loadmat('filepath to male Spots.mat')
fem1 = scipy.io.loadmat('filepath to female Spots.mat')

I have a function here that extracts the intensity of each fluorescent spot of each frame of the MS2 or PP7 movie

In [ ]:
def getIntensity(mat): ##this works better
    """gets spot intensity from each spot of each frame of MS2 movie"""
    spots = mat['Spots']
    nFrames = spots[0].shape[0]
    #print(nFrames)
    FixedAreaIntensity3 = []
    for frame in range(0, nFrames-1):
        FixedAreaIntensity3.append([])
        if len(spots[0][frame][0]) > 0:
            nSpots = spots[0][frame][0][0].shape[0]
                #print(nSpots)
            for spot in range(0, nSpots):
                val = spots[0][frame][0][0][spot][16][0][0]
                FixedAreaIntensity3[frame].append(val)
    return FixedAreaIntensity3 

In [ ]:
#Save intensities for a given embryo as a variable
IntMale1 = getIntensity(male1) 
IntFem1 = getIntensity(fem1) 

To plot male and female embryos on the same strip plot, use the following code:

In [ ]:
sea.set(rc={'figure.figsize':(30,10)})
sea.set_style("white")
sea.set(font_scale = 1.5)
fig, ax = plt.subplots()

dx, dy = 8/72., 0.
offset = transforms.ScaledTranslation(dx, dy, ax.figure.dpi_scale_trans)
shadow_transform = ax.transData + offset

sea.stripplot(data = IntFem1, color=sea.xkcd_rgb["orange"])
sea.stripplot(data = IntMale1, color=sea.xkcd_rgb["dark cyan"], transform=shadow_transform)
plt.ylabel("Fluorescent Spot Intensity", fontsize = 20)
plt.xlabel("Frame", fontsize = 20)
sea.set(font_scale = 16)
plt.title('Fluorescent Spot Intensities By Frame', fontsize = 32)
plt.legend(labels=["M","F"], title = "Sex", fontsize = '26', title_fontsize = "30")
plt.ylim(0,1200000)

#plt.savefig("MvF.png", dpi= 300)

To plot the distribution of fluorescent intensity of many embryos, I made a list of values for all males and all female embryos

In [ ]:
f_all = []
f_all.append(f1)
f_all.append(f2)

m_all = []
m_all.append(m1)
m_all.append(m2)

f_all = np.concatenate(f_all)
m_all = np.concatenate(m_all)

Then I could plot M v F histograms like so:

In [ ]:
fig, ax = plt.subplots()
ax.set(yscale="log")
sea.set_style("white")
sea.histplot(f_all, color=sea.xkcd_rgb["orange"], kde = True, stat = "density")
plt.xlim(0,650000)
sea.set(font_scale = 0.75)
plt.ylabel("Density", fontsize = 14)
plt.xlabel("Fluorescent Spot Intensity", fontsize = 14)
plt.title('Distribution of Fluorescent Spot Intensities of Female Loci', fontsize = 16)
plt.legend(labels=["M","F"], title = "Sex", fontsize = '26', title_fontsize = "30")

plt.savefig("Fvalues.png", dpi= 300)

## Plotting particle traces instead of single fluorescent spot traces

A particle represents a transcribing locus tracked through time. In order to look at the average fluorescent particle intensity for _PP7-btd_ in male and female embryos, I manually curated the particle tracking in the custom image processing pipeline and exported the data to a .csv file. From there, I plotted the data as follows:

In [ ]:
df = pd.read_csv('filepath to your .csv file containing particle intensity values')

In [ ]:
sea.set_palette("Set2")
sea.set_style("white")
sea.lineplot(data=df, x="Frame", y="Fluorescent Particle Intensity", hue="Sex")
plt.ylabel("Fluorescent Particle Intensity", fontsize = 12)
plt.xlabel("Frame", fontsize = 12)
plt.title('Average Particle Intensity', fontsize = 10)
plt.savefig("particleTraces", dpi=300)